In [17]:
from collections import OrderedDict
from pathlib import Path

import mne
from mne_bids import (BIDSPath, read_raw_bids, write_raw_bids)
import numpy as np

import IO

In [24]:
# Enter filename and BIDS root

#bids_root = Path('/Users/richardkoehler/Documents/Neurology_Data/BIDS Berlin/')
#bids_root = "C:\\Users\\ICN_admin\\Charité - Universitätsmedizin Berlin\\Interventional Cognitive Neuromodulation - Data\\Datasets\\BIDS Berlin"
bids_root = r"C:\Users\ICN_admin\Charité - Universitätsmedizin Berlin\Interventional Cognitive Neuromodulation - Data\Datasets\BIDS Pittsburgh"
#filename = 'sub-002_ses-20200131_task-selfpacedrotation202001310001_run-4_ieeg'
filename = 'sub-000_ses-right_task-force_run-0_ieeg'

subject, session, task, run = IO.get_subject_sess_task_run(filename)
dataype = 'ieeg'
bids_path = BIDSPath(subject=subject, session=session, task=task, run=run, datatype=dataype, root=bids_root)

In [25]:
task

'force'

In [26]:
bids_path

BIDSPath(
root: C:\Users\ICN_admin\Charité - Universitätsmedizin Berlin\Interventional Cognitive Neuromodulation - Data\Datasets\BIDS Pittsburgh
datatype: ieeg
basename: sub-000_ses-right_task-force_run-0)

In [27]:
# If preload is set to TRUE, write_raw_bids might not work. Only load_data if necessary.
raw = read_raw_bids(bids_path=bids_path, extra_params=dict(preload=False), verbose=False)

Extracting parameters from C:\Users\ICN_admin\Charité - Universitätsmedizin Berlin\Interventional Cognitive Neuromodulation - Data\Datasets\BIDS Pittsburgh\sub-000\ses-right\ieeg\sub-000_ses-right_task-force_run-0_ieeg.vhdr...
Setting channel info structure...
Reading channel info from C:\Users\ICN_admin\Charité - Universitätsmedizin Berlin\Interventional Cognitive Neuromodulation - Data\Datasets\BIDS Pittsburgh\sub-000\ses-right\ieeg\sub-000_ses-right_task-force_run-0_channels.tsv.
Reading electrode coords from C:\Users\ICN_admin\Charité - Universitätsmedizin Berlin\Interventional Cognitive Neuromodulation - Data\Datasets\BIDS Pittsburgh\sub-000\ses-right\ieeg\sub-000_ses-right_task-force_run-0_space-mni_electrodes.tsv.


<ipython-input-27-3c1f71791417>:2: RuntimeWarning: Did not find any events.tsv associated with sub-000_ses-right_task-force_run-0.

The search_str was "C:\Users\ICN_admin\Charité - Universitätsmedizin Berlin\Interventional Cognitive Neuromodulation - Data\Datasets\BIDS Pittsburgh\sub-000\**\sub-000_ses-right*events.tsv"
  raw = read_raw_bids(bids_path=bids_path, extra_params=dict(preload=False), verbose=False)
<ipython-input-27-3c1f71791417>:2: RuntimeWarning: There are channels without locations (n/a) that are not marked as bad: ['MOV_RIGHT', 'MOV_LEFT', 'MOV_RIGHT_CLEAN', 'MOV_LEFT_CLEAN']
  raw = read_raw_bids(bids_path=bids_path, extra_params=dict(preload=False), verbose=False)
<ipython-input-27-3c1f71791417>:2: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw = read_raw_bids(bids_path=bids_path, extra_params=dict(preload=False), verbose=False)


In [30]:
mne.channels.make_dig_montage?

In [32]:
elec / 1000

array([[ 0.03731817, -0.04861013,  0.06179765],
       [ 0.04015989, -0.03731593,  0.06431172],
       [ 0.04094304, -0.02721778,  0.06409518],
       [ 0.03978396, -0.01700523,  0.06386618],
       [ 0.03968814, -0.00552802,  0.06168254],
       [ 0.03751916,  0.00430491,  0.06054126],
       [        nan,         nan,         nan],
       [        nan,         nan,         nan],
       [        nan,         nan,         nan],
       [        nan,         nan,         nan]])

In [33]:
# Read in file containing electrode locations
electr_file = bids_path.directory / 'sub-000_electrodes.tsv'

data = np.loadtxt(electr_file, dtype=str, delimiter='\t', comments=None, encoding='utf-8')
column_names = data[0, :]
info = data[1:, :]

electrode_tsv = OrderedDict()
for i, name in enumerate(column_names):
    electrode_tsv[name] = info[:, i].tolist()

# Load in channel names
ch_names = electrode_tsv['name']

# Load in the xyz coordinates as a float
elec = np.empty(shape=(len(ch_names), 3))
for ind, axis in enumerate(['x', 'y', 'z']):
    elec[:, ind] = list(map(float, electrode_tsv[axis]))

# Create mne montage
montage = mne.channels.make_dig_montage(ch_pos=dict(zip(ch_names, elec / 1000)), coord_frame='mni_tal')
print('Created %s channel positions' % len(ch_names))
print(dict(zip(ch_names, elec)))

# Set montage. Warning is issued if channels don't match. Consider getting locations for missing channels.
raw.set_montage(montage, on_missing='warn')

Created 10 channel positions
{'ECOG_RIGHT_0': array([ 37.318174  , -48.61012664,  61.79765474]), 'ECOG_RIGHT_1': array([ 40.1598943 , -37.31592983,  64.31171618]), 'ECOG_RIGHT_2': array([ 40.94303578, -27.21778456,  64.09518408]), 'ECOG_RIGHT_3': array([ 39.78395522, -17.00523081,  63.86618136]), 'ECOG_RIGHT_4': array([39.68813641, -5.52802457, 61.68254254]), 'ECOG_RIGHT_5': array([37.51915924,  4.30491341, 60.54126355]), 'STN_RIGHT_0': array([ 11.82808023, -15.15974827,  -7.76909846]), 'STN_RIGHT_1': array([ 12.3908352 , -14.29250828,  -5.9367987 ]), 'STN_RIGHT_2': array([ 12.93084915, -13.41284994,  -4.09165662]), 'STN_RIGHT_3': array([ 13.44457579, -12.52669693,  -2.24263538])}


<ipython-input-33-c8e7ea4517db>:26: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw.set_montage(montage, on_missing='warn')
<ipython-input-33-c8e7ea4517db>:26: RuntimeWarning: DigMontage is only a subset of info. There are 4 channel positions not present in the DigMontage. The required channels are:

['MOV_RIGHT', 'MOV_LEFT', 'MOV_RIGHT_CLEAN', 'MOV_LEFT_CLEAN'].

Consider using inst.set_channel_types if these are not EEG channels, or use the on_missing parameter if the channel positions are allowed to be unknown in your analyses.
  raw.set_montage(montage, on_missing='warn')


<RawBrainVision | sub-000_ses-right_task-force_run-0_ieeg.eeg, 10 x 130001 (130.0 s), ~31 kB, data not loaded>

In [34]:
# Write out files in BIDS format
# Might issue SameFileError if no changes are made to raw. Can be ignored, since _ieeg files don't need to be overwritten.

# Workaround, if raw data has been loaded.
if raw.preload == True:
    fname_fif = bids_path.directory / (bids_path.basename + 'raw.fif')
    raw.save(fname_fif, proj=True, overwrite=True)
    raw = mne.io.read_raw_fif(fname_fif, preload=False, verbose=False)

write_raw_bids(raw, bids_path, overwrite=True, verbose=True)

Extracting parameters from C:\Users\ICN_admin\Charité - Universitätsmedizin Berlin\Interventional Cognitive Neuromodulation - Data\Datasets\BIDS Pittsburgh\sub-000\ses-right\ieeg\sub-000_ses-right_task-force_run-0_ieeg.vhdr...
Setting channel info structure...

Writing 'C:\Users\ICN_admin\Charité - Universitätsmedizin Berlin\Interventional Cognitive Neuromodulation - Data\Datasets\BIDS Pittsburgh\participants.tsv'...

participant_id	age	sex	center_ID	disease	center	curator	UPDRS_total	UPDRS_left	UPDRS_right	line_noise	Length_of_diagnosis_prior_to_surgery	hand
sub-000	n/a	F	DBS4017	PD	UPMC	Richardson	28	11	9	60	10.7	n/a
sub-001	51.2	M	DBS4029	PD	UPMC	Richardson	27	5	7	60	14	n/a
sub-002	63.8	M	DBS4011	PD	UPMC	Richardson	40	10	16	60	4.5	n/a
sub-003	54.7	M	DBS4020	PD	UPMC	Richardson	n/a	n/a	n/a	60	21.8	n/a
sub-004	53.8	M	DBS4033	PD	UPMC	Richardson	33	13	13	60	7.2	n/a

Writing 'C:\Users\ICN_admin\Charité - Universitätsmedizin Berlin\Interventional Cognitive Neuromodulation - Data\Datasets\B

<ipython-input-34-91edfaef2876>:10: RuntimeWarning: No events found or provided. Please make sure to set channel type using raw.set_channel_types or provide events_data.
  write_raw_bids(raw, bids_path, overwrite=True, verbose=True)


ValueError: Previous BIDS version used, please redo the conversion to BIDS in a new directory after ensuring all software is updated